<a href="https://colab.research.google.com/github/martinpovolny/colaboratory/blob/master/Cars_vs_trucks_hyperparam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we download and unzip the data. The data comes in sorted into subdirectories by categories.

For sorting I first used: https://github.com/martinpovolny/clsf and manually fixed bugz.

In [0]:
!wget http://ujc.hmpf.cz/download/cars-sorted-2019-06-01.zip  -O /tmp/cars-sorted.zip

--2019-06-01 12:39:42--  http://ujc.hmpf.cz/download/cars-sorted-2019-06-01.zip
Resolving ujc.hmpf.cz (ujc.hmpf.cz)... 147.231.205.34
Connecting to ujc.hmpf.cz (ujc.hmpf.cz)|147.231.205.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66101608 (63M) [application/zip]
Saving to: ‘/tmp/cars-sorted.zip’

/tmp/cars-sorted.zi 100%[===================>]  63.04M  11.2MB/s    in 6.6s    

2019-06-01 12:39:49 (9.59 MB/s) - ‘/tmp/cars-sorted.zip’ saved [66101608/66101608]



In [0]:
import os
import zipfile

local_zip = '/tmp/cars-sorted.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/cars-sorted')
zip_ref.close()

In [0]:
train_dir = os.path.join('/tmp/cars-sorted/sorted')


I am downscaling the images to 90x160. Seems to be enough to tell car from a truck and front from rear.

In [0]:
image_size=(90, 160)

Common approach (tutorials, courses) is having the train and validation data in separate directories. I have the data from the same source and would have to split it manually or write some code to split it to use that approach.

So for having train and validation data in the same directory I am using this: https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
# FIXME: loss_weights?
batch_size = 128

train_datagen = ImageDataGenerator(rescale=1./255,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data



Found 3578 images belonging to 5 classes.
Found 893 images belonging to 5 classes.


In [0]:
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf


In [0]:
HP_DENSE_UNITS = hp.HParam('dense_units', hp.Discrete([64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.5))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_DENSE_UNITS, HP_DROPOUT], # , HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )
  
log_dir = "logs/hparam_tuning"

In [0]:
import numpy as np

from tensorflow.keras.models import Sequential;
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation;

In [0]:
def model_4(image_size, num_classes, hparams):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(image_size[0], image_size[1], 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(hparams[HP_DENSE_UNITS]))
    model.add(Activation('relu'))
    model.add(Dropout(rate = hparams[HP_DROPOUT]))
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))

    return model


In [0]:
from tensorflow.keras.optimizers import Adam, RMSprop

#model.compile(loss='binary_crossentropy',
#              optimizer=RMSprop(lr=0.001),
#              metrics=['acc'])

def compile_model(model, hparams):
    opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(opt,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        loss_weights=None,
        sample_weight_mode=None,
        weighted_metrics=None,
        target_tensors=None)
    
    return model



In [0]:

def fit_model(model, hparams):
    history = model.fit_generator(
          train_generator,
          steps_per_epoch=8,  
          epochs=15,
          validation_data = validation_generator, 
          validation_steps = validation_generator.samples // batch_size,
          verbose=1,
          callbacks=[
              # tf.keras.callbacks.TensorBoard(logdir),  # log metrics
              hp.KerasCallback(log_dir, hparams),  # log hparams
          ]
    )
    
    return history

In [0]:
def train_test_model(hparams):
    model = model_4(image_size, 5, hparams)
    model = compile_model(model, hparams)
    history = fit_model(model, hparams)
    # _, accuracy = model.evaluate(x_test, y_test) # FIXME x_test, y_test
    
    accuracy = history.history['val_accuracy'][-1]
    return accuracy

Simple Hyperparameter search using Grid search according to https://www.tensorflow.org/tensorboard/r2/hyperparameter_tuning_with_hparams

In [42]:
session_num = 0

for dense_units in HP_DENSE_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value): # FIXME: testing just min and max
    #for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_DENSE_UNITS: dense_units,
          HP_DROPOUT: dropout_rate,
          #HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      
      train_test_model(hparams)
      session_num += 1

--- Starting trial: run-0
{'dense_units': 64, 'dropout': 0.1}
Epoch 1/15
8/8 [==============================] - 21s 3s/step - loss: 1.3445 - accuracy: 0.4600 - val_loss: 1.2651 - val_accuracy: 0.4740
Epoch 2/15
8/8 [==============================] - 20s 2s/step - loss: 1.2363 - accuracy: 0.5413 - val_loss: 1.1731 - val_accuracy: 0.5156
Epoch 3/15
8/8 [==============================] - 19s 2s/step - loss: 1.0411 - accuracy: 0.6035 - val_loss: 1.0251 - val_accuracy: 0.6484
Epoch 4/15
8/8 [==============================] - 21s 3s/step - loss: 0.8550 - accuracy: 0.6680 - val_loss: 1.1143 - val_accuracy: 0.5938
Epoch 5/15
8/8 [==============================] - 19s 2s/step - loss: 0.8225 - accuracy: 0.7051 - val_loss: 0.8341 - val_accuracy: 0.7370
Epoch 6/15
8/8 [==============================] - 19s 2s/step - loss: 0.6783 - accuracy: 0.7422 - val_loss: 0.8474 - val_accuracy: 0.7279
Epoch 7/15
8/8 [==============================] - 20s 2s/step - loss: 0.6537 - accuracy: 0.7515 - val_loss: 0.

best results: ~80% val_accuracy, 128 dense units, dropout either 0.1 or 0.5